In [1]:
DEVICE_NAME = { 'BC 044 - Piedecuesta',
 'BC 061 - Carrera Primera',
 'BC 185 - Llano Grande Palmira',
 'BC 197 - Guatapuri',
 'BC 205 - Villa Colombia',
 'BC 210 - Banca Colombia Cartagena',
 'BC 216 - Sabana de Torres',
 'BC 253 - Puerta del Norte',
 'BC 291 - Las Palmas',
 'BC 302 - Quebrada Seca',
 'BC 306 - Barrancabermeja',
 'BC 311 - Bello',
 'BC 371 - Caucasia',
 'BC 398 - Mariquita',
 'BC 424 - Honda',
 'BC 453 - Pitalito',
 'BC 454 - Quinta Avenida 2',
 'BC 459 - Campo Alegre',
 'BC 495 - El Bosque',
 'BC 496 - Iwanna',
 'BC 514 - Centro Comercial Único',
 'BC 523 - Olímpica',
 'BC 524 - S.A. Valledupar',
 'BC 534 - Buganviles',
 'BC 613 - La America',
 'BC 656 - Mayales',
 'BC 659 - Girardot',
 'BC 66 - Palmira',
 'BC 678 - Paseo de la Castellana',
 'BC 705 - El Bagre',
 'BC 749 - Santa Monica',
 'BC 750 - Roosevelt',
 'BC 764 - Jamundí',
 'BC 776 - Lebrija',
 'BC 78 - El Cacique',
 'BC 784 - Centro Colon',
 'BC 787 - Bocagrande Carrera Tercera',
 'BC 789 - Manga',
 'BC 792 - Paseo del comercio',
 'BC 793 - Profesionales',
 'BC 796 - Girón',
 'BC 799 - Floridablanca',
 'BC 816 - Calle 10',
 'BC 821 GRANADA CALI',
 'BC 824 - Ventura Plaza',
 'BC 825 - Astrocentro',
 'BC 829 - Unicentro Cali',
 'BC 834 - San Mateo',
 'BC 863 - Los Patios',
 'BC 88 - Cúcuta',
 'BC 90 - Megamall'}


label = {"aa-consumo-activa", "front-consumo-activa"}

In [2]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import datetime as dt
import json
import locale
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import pyppdf.patch_pyppeteer

pio.renderers.default = "notebook"
pio.templates.default = "plotly_white"


# this enables relative path imports
import os
from dotenv import load_dotenv
load_dotenv()
_PROJECT_PATH: str = os.environ["_project_path"]
_PICKLED_DATA_FILENAME: str = os.environ["_pickled_data_filename"]

import sys
from pathlib import Path
project_path = Path(_PROJECT_PATH)
sys.path.append(str(project_path))

import config_v2 as cfg

from library_report_v2 import Cleaning as cln
from library_report_v2 import Graphing as grp
from library_report_v2 import Processing as pro
from library_report_v2 import Configuration as repcfg

In [3]:
def show_response_contents(df):
    print("The response contains:")
    print(json.dumps(list(df['variable'].unique()), sort_keys=True, indent=4))
    print(json.dumps(list(df['device'].unique()), sort_keys=True, indent=4))

month_name = cfg.MONTH_NAME

In [5]:
df_info = pd.read_excel(project_path / 'tools' / "AMH Sedes BC.xlsx")

df = pd.read_pickle(project_path / 'data' / _PICKLED_DATA_FILENAME)
#df = df.query("device_name == DEVICE_NAME")

In [6]:
df_info = pd.read_excel(project_path / 'tools' / "AMH Sedes BC.xlsx")

df = pd.read_pickle(project_path / 'data' / _PICKLED_DATA_FILENAME)
#df = df.query("device_name == DEVICE_NAME")
df = df[df['variable'].isin(label)]

# Legacy code (including the library) expects these column names
# but the new Ubidots library returns more specific column names
# so renaming is necessary. TODO: rework the Report library
# so that it uses these more descriptive column names.
df = df.rename(columns={'variable_label':'variable','device_label':'device',})

show_response_contents(df)

The response contains:
[
    "front-consumo-activa",
    "aa-consumo-activa"
]
[
    "bc14",
    "bc45",
    "bc71",
    "bc2",
    "bc60",
    "bc---floridablanca",
    "bc30",
    "bc44",
    "bc56",
    "bc33",
    "bc-santa-monica",
    "bc41",
    "bc35",
    "bc57",
    "bc11",
    "bc01-iluminacion-girardot",
    "bc---s.a.-valledupar",
    "bc-291-las-palmas",
    "bc12",
    "bc21",
    "bc10",
    "bc-megamall",
    "bc8",
    "bc31",
    "bc16",
    "bc32",
    "bc54",
    "bc26",
    "bc58t",
    "bc---s.a.-granada",
    "bc4",
    "bc38",
    "bc50t",
    "bc46",
    "bc65",
    "bc64",
    "bc42",
    "bc43",
    "bc71-2",
    "bc18",
    "bc27",
    "bc40",
    "bc36",
    "bc61",
    "bc15",
    "bc-816-1---calle-10",
    "bc7",
    "bc49",
    "gateway-1-bc-uc",
    "bc37",
    "bc6"
]


In [15]:
df = df.sort_values(by=['variable','datetime'])
df = pro.datetime_attributes(df)

df_bl, df_st = pro.split_into_baseline_and_study(df, baseline=cfg.BASELINE, study=cfg.STUDY, inclusive='left')

study_daterange = pd.Series(pd.date_range(start=cfg.STUDY[0], end=cfg.STUDY[1], freq='D'))

cargas = df_st[df_st["variable"].isin(cfg.ENERGY_VAR_LABELS)].copy()
front = df_st[df_st["variable"].isin(['front-consumo-activa'])].copy()

cargas = cln.remove_outliers_by_zscore(cargas, zscore=4)
front = cln.remove_outliers_by_zscore(front, zscore=4)

In [26]:
cargas_hour = cargas.groupby(by=["variable",'device_name']).resample('1h').sum().round(2).reset_index().set_index('datetime')
cargas_hour = pro.datetime_attributes(cargas_hour)

cargas_day = cargas.groupby(by=["variable",'device_name']).resample('1D').sum().reset_index().set_index('datetime')
cargas_day = pro.datetime_attributes(cargas_day)

cargas_month = cargas.groupby(by=["variable",'device_name']).resample('1M').sum().reset_index().set_index('datetime')
cargas_month = pro.datetime_attributes(cargas_month)

front_hour = front.groupby(by=["variable",'device_name']).resample('1h').sum().round(2).reset_index().set_index('datetime')
front_hour = pro.datetime_attributes(front_hour)

front_day = front.groupby(by=["variable",'device_name']).resample('1D').sum().reset_index().set_index('datetime')
front_day = pro.datetime_attributes(front_day)

front_month = front.groupby(by=["variable",'device_name']).resample('1M').sum().reset_index().set_index('datetime')
front_month = pro.datetime_attributes(front_month)


In [51]:
front_month_x2 = front_month.loc[:,("variable","device_name","value","month")]
front_month_x2['value'] = front_month_x2['value'].round(2) 

In [61]:
df_front_cargas = pd.concat([front, cargas])
cargas_nighttime_cons = df_front_cargas[df_front_cargas["hour"].isin(cfg.NIGHT_HOURS)].copy()

In [ ]:
front_month = front.groupby(by=["variable",'device_name']).resample('1M').sum().reset_index().set_index('datetime')
front_month = pro.datetime_attributes(front_month)

In [88]:
df_front_cargas = pd.concat([front, cargas])

cargas_nighttime_cons = df_front_cargas[df_front_cargas["hour"].isin(cfg.NIGHT_HOURS)].copy()
cargas_nighttime_cons = pro.datetime_attributes(cargas_nighttime_cons)

cargas_daily_nighttime_cons = (
    cargas_nighttime_cons
    .groupby(['variable','day'])['value']
    .sum()
    .to_frame()
)

In [89]:
cargas_nighttime_cons_x2 = cargas_nighttime_cons.groupby(by=["variable",'device_name']).resample('1M').sum().reset_index().set_index('datetime')
cargas_nighttime_cons_x2 = pro.datetime_attributes(cargas_nighttime_cons_x2)

cargas_nighttime_cons_x2 = cargas_nighttime_cons_x2.loc[:,("variable","device_name","value","month")]
cargas_nighttime_cons_x2['value'] = cargas_nighttime_cons_x2['value'].round(2) 


In [94]:
front_month_x2.to_clipboard()
#cargas_nighttime_cons_x2.to_clipboard()